## Simpler code with basics of semi-mechanistic approach
Again, thinking about this equation in Faria, et al:
$\\i_{s,t} = (1-\frac{n_{s,t}}{N})R_{s,t}\sum_{\tau<t} i_{s,\tau}g_{t-\tau}$

And again reducing the complexity of this by ignoring strains,
we'll now consider the equation with susceptible depletion included:
$\\i_t = (1-\frac{n_t}{N})R_t\sum_{\tau<t} i_{\tau}g_{t-\tau}$

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'

In [ ]:
def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

In [ ]:
# Model parameters
seed = 1.0
r0 = 8.0
gen_time_sd = 1.5
gen_time_mean = 5.0

In [ ]:
# Generation time
times = 40
gamma_params = get_gamma_params_from_mean_sd(gen_time_mean, gen_time_sd)
gen_time_densities = np.diff(gamma.cdf(range(times + 1), **gamma_params))
pd.Series(gen_time_densities, index=range(times)).plot()

In [ ]:
inc = np.zeros(times)
inc[0] = seed
pop = 100.0
suscept = pop - seed
for t in range(1, times):
    this_inc = (inc[:t] * gen_time_densities[t-1::-1]).sum() * r0 * suscept / pop
    inc[t] = this_inc
    suscept = max(suscept - this_inc, 0.0)

In [ ]:
pd.Series(inc).plot(labels={'index': 'day', 'value': 'incidence'})

Now with susceptible depletion, we have an epi-curve that goes up in the initial phase with $R_0 > 1$,
but comes back down as susceptibles are depleted and so $R_t$ falls below one.